In [9]:
import rasterio 
import numpy
from owslib.wfs import WebFeatureService 
from rasterio.coords import BoundingBox
import geopandas as gpd
import io

In [6]:
geotiff_path = "../../data/R_25DN2.TIF"

with rasterio.open(geotiff_path) as src:
    bounds = src.bounds  # BoundingBox(left, bottom, right, top)
    crs = src.crs  

print(f"GeoTIFF Bounds: {bounds}")
print(f"GeoTIFF CRS: {crs}")

GeoTIFF Bounds: BoundingBox(left=115000.0, bottom=481250.0, right=120000.0, top=487500.0)
GeoTIFF CRS: EPSG:28992


In [7]:
bbox_rd_new = (bounds.left, bounds.bottom, bounds.right, bounds.top)

In [28]:
wfs_bag = 'https://service.pdok.nl/lv/bag/wfs/v2_0'
layer = 'bag:pand'

In [40]:
# Connect to the WFS server

wfs = WebFeatureService(url=wfs_bag, version='2.0.0', parse_remote_metadata=False)
srs = 'EPSG:28992' 

capabilities = wfs.getcapabilities()

response = wfs.getfeature(typename=layer,
                          bbox=bbox_rd_new + (srs,),  # Include EPSG code for RD New
                          outputFormat='application/json; subtype=geojson')

response_text = response.read()

print(response_text[:500])
gdf = gpd.read_file(io.BytesIO(response.read()))

print(gdf.head())

b'{\n"type": "FeatureCollection",\n"numberMatched": 4,\n"name": "woonplaats",\n"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:EPSG::28992" } },\n"features": [\n{ "type": "Feature", "id": "woonplaats.fdc25bab-8fcc-43c0-94a1-dfe51cad0801", "properties": { "identificatie": "1050", "rdf_seealso": "http:\\/\\/bag.basisregistraties.overheid.nl\\/bag\\/id\\/woonplaats\\/1050", "status": "Woonplaats aangewezen", "woonplaats": "Amstelveen" }, "bbox": [ 114538.296, 472779.61, 122404.544, 482651.756 ]'
                                                id identificatie  \
0  woonplaats.fdc25bab-8fcc-43c0-94a1-dfe51cad0801          1050   
1  woonplaats.ac84bad8-184e-46df-be5d-bd54b073dec8          1617   
2  woonplaats.3df401f1-be85-4c75-8307-88db35bd85ff          1618   
3  woonplaats.9acef35a-7d3a-4f7a-82b2-22ad5260cf1e          3594   

                                         rdf_seealso                 status  \
0  http://bag.basisregistraties.overheid.nl/bag/i...  Woonplaats

In [37]:
# Check the GeoDataFrame
# print(gdf.head())

geometry_counts = gdf.geometry.geom_type.value_counts()
print("Geometry type counts:")
print(geometry_counts)

Geometry type counts:
MultiPolygon    4
Name: count, dtype: int64
